## MultiDiGraph

Experiments in "collapsing" a multidigraph into a simple probability-weighted digraph.

Meaning of interactions needs to be gauged. Potential for ML. 

Desired features:
+ "Reasonable" distribution of final weights
+ "Meaningful" weights, as a function of incoming edges and total stats

Before:             |  After:
:-------------------------:|:-------------------------:
![Image](Report/Figures/MultiDiGraph.png)  |  ![Image](Report/Figures/MultiDiGraphII.png)


### Weighted Cascade

__IC-WC__: IC model with weighted cascade probabilities, that is, each in-coming arc of $v$ has probability $\frac{1}{d_v}$, where $d_v$ is the in-degree of $v$ before removing duplicated arcs. If there are $c(u,v)$ arcs from $u$ to $v$, we have $$p_{uv} = 1 - (1 - 1/d(v))^{c(u,v)}$$ in the graph with duplicated arcs removed.

In [1]:
def create_multidigraph():
    G = nx.MultiDiGraph()
    G.add_nodes_from(["Peter", "Paul", "Mary"])
    G.add_edge("Peter", "Mary", kind="FR")
    G.add_edge("Peter", "Mary", kind="MT")
    G.add_edge("Mary", "Peter", kind="RE")
    G.add_edge("Mary", "Paul", kind="FR")
    G.add_edge("Mary", "Paul", kind="RT")
    G.add_edge("Peter", "Paul", kind="RT")
    G.add_edge("Peter", "Paul", kind="RT")
    G.add_edge("Peter", "Paul", kind="MT")
    return G
    
def to_digraph(M):
    """Combines parallel edges in M to weighted edges in G"""
    
    # First, cast to DiGraph. This arbitrarily chooses attributes of paralell edges
    G = nx.DiGraph(M)
    
    # Then, count in-edges and 'kinds'
    
    #for u,v,data in M.edges(data=True):
    
    
    # Remove all attributes from edges
    for u,v,data in M.edges(data=True):
        for k in data:
            if k in G[u][v]: #why?
                del G[u][v][k]
            
    nx.set_edge_attributes(G, 0.0, "weight")
    
    # Now start building the weight = p_{uv}'s of G 
    for u,v,data in M.edges(data=True):
        G[u][v]["weight"] += weights[data["kind"]]
    
    return G.reverse(copy=False)

def print_multidigraph(G):
    for u,v,data in G.edges(data=True):
        print(u, v, data)

In [2]:
import networkx as nx

weights = {"RT" : 0.01, "RE" : 0.001, "RT" : 0.01, "MT" : 0.01, "FR" : 0.01}

#for u, v, keys, weight in G.edges(data='weight', keys=True):
#    if weight is not None:
#        pass

def num_edges_from_to(G, u, v):
    """compute c(u, v)"""
    
    return len([x for x in G.predecessors(u) if x == v])

def weighted_edges_from_to(G, u, v):
    """compute W(u, v)"""
    weight = 1.0
    
    
    print(list(G.get_edge_data(u,v)))
#    for item in G[u][v]:
 #       print(item)
  #      weight += 1
        #weight += weights[G[u][v]["kind"]]
                
    return weight
    

G = create_multidigraph()

#print(weighted_edges_from_to(G, "Peter", "Paul"))
#print(weighted_edges_from_to(G, "Paul", "Peter"))

print("=====MULTIDIGRAPH:=====")
print_multidigraph(G)
G = to_digraph(G)

print("=====CONVERT TO DIGRAPH:=====")
print_multidigraph(G)


=====MULTIDIGRAPH:=====
Peter Mary {'kind': 'FR'}
Peter Mary {'kind': 'MT'}
Peter Paul {'kind': 'RT'}
Peter Paul {'kind': 'RT'}
Peter Paul {'kind': 'MT'}
Mary Peter {'kind': 'RE'}
Mary Paul {'kind': 'FR'}
Mary Paul {'kind': 'RT'}
=====CONVERT TO DIGRAPH:=====
Peter Mary {'weight': 0.01}
Paul Peter {'weight': 0.30000000000000004}
Paul Mary {'weight': 0.2}
Mary Peter {'weight': 0.2}


In [4]:
print(list(G.in_edges))

print(G.get_edge_data("Paul", "Peter"))

print(list(G["Paul"]))

print(list(G.predecessors("Peter")))

[('Mary', 'Peter', 0), ('Mary', 'Paul', 0), ('Mary', 'Paul', 1), ('Peter', 'Paul', 0), ('Peter', 'Paul', 1), ('Peter', 'Paul', 2), ('Peter', 'Mary', 0), ('Peter', 'Mary', 1)]
None
[]
['Mary']
